Compare LSTM with other methods: CNN, MLP, CNN-LSTM

In [0]:
import numpy as np
import pandas as pd 
import os

In [0]:
#loading data 
os.listdir('../input')
sales_data = pd.read_csv('../input/sales_train.csv')
item_cat = pd.read_csv('../input/item_categories.csv')
items = pd.read_csv('../input/items.csv')
shops = pd.read_csv('../input/shops.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')
test_data = pd.read_csv('../input/test.csv')


In [0]:
def basic_eda(df):
    print("----------TOP 5 RECORDS--------")
    print(df.head(5))
    print("----------INFO-----------------")
    print(df.info())
    print("----------Describe-------------")
    print(df.describe())
    print("----------Columns--------------")
    print(df.columns)
    print("----------Data Types-----------")
    print(df.dtypes)
    print("-------Missing Values----------")
    print(df.isnull().sum())
    print("-------NULL values-------------")
    print(df.isna().sum())
    print("-----Shape Of Data-------------")
    print(df.shape)
    
    

In [0]:
#Litle bit of exploration of data

print("=============================Sales Data=============================")
basic_eda(sales_data)
print("=============================Test data=============================")
basic_eda(test_data)
print("=============================Item Categories=============================")
basic_eda(item_cat)
print("=============================Items=============================")
basic_eda(items)
print("=============================Shops=============================")
basic_eda(shops)
print("=============================Sample Submission=============================")
basic_eda(sample_submission)



In [0]:

sales_data['date'] = pd.to_datetime(sales_data['date'],format = '%d.%m.%Y')

In [0]:

dataset = sales_data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')


In [0]:
dataset.reset_index(inplace = True)

In [0]:
dataset.head()
dataset.shape

In [0]:

dataset = pd.merge(test_data,dataset,on = ['item_id','shop_id'],how = 'left')

In [0]:

dataset.fillna(0,inplace = True)
dataset.head()

In [0]:
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

In [0]:
dataset.shape

In [0]:
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
y_train = dataset.values[:,-1:]

X_test = np.expand_dims(dataset.values[:,1:],axis = 2)
print(X_train.shape,y_train.shape,X_test.shape)


In [0]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [0]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [0]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [0]:
X_train_mlp = dataset.values[:,:-1]
y_train_mlp = dataset.values[:,-1:]

X_test_mlp = dataset.values[:,1:]

print(X_train_mlp.shape,y_train_mlp.shape,X_test_mlp.shape)

In [0]:
model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X_train_mlp.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)
model_mlp.summary()

In [0]:
X_train_mlp.shape

In [0]:
mlp_history = model_mlp.fit(X_train_mlp,y_train_mlp,batch_size = 4096,epochs = 10)

In [0]:
submission_mlp = model_mlp.predict(X_test_mlp)

In [0]:
submission_mlp

In [0]:
submission_mlp = submission_mlp.clip(0,20)
submission_mlp_csv = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_mlp.ravel()})

submission_mlp_csv.to_csv('sub_mlp.csv',index = False)


CNN

In [0]:
print(X_train.shape,y_train.shape,X_test.shape)

In [0]:
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer=adam)
model_cnn.summary()

In [0]:
cnn_history = model_cnn.fit(X_train, y_train,batch_size = 4096,epochs = 10)

In [0]:
submission_cnn = model_cnn.predict(X_test)


In [0]:
submission_cnn

In [0]:
submission_cnn = submission_cnn.clip(0,20)
# creating dataframe with required columns 
submission_cnn_csv = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_cnn.ravel()})

# creating csv file from dataframe
submission_cnn_csv.to_csv('sub_cnn.csv',index = False)

In [0]:
submission_cnn_csv.head()

LSTM

In [0]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mse', optimizer=adam)
model_lstm.summary()

In [0]:
model_lstm.fit(X_train,y_train,batch_size = 4096,epochs = 50)

In [0]:
submission_lstm = model_lstm.predict(X_test)

In [0]:
submission_lstm = submission_lstm.clip(0,20)
submission_lstm_csv = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_lstm.ravel()})

# creating csv file from dataframe
submission_lstm_csv.to_csv('sub_lstm.csv',index = False)

CNN-LSTM on train and validation

In [0]:

X_test.shape

In [0]:
subsequences = 3
timesteps = X_train.shape[1]//subsequences
X_train_series_sub = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_test_series_sub = X_test.reshape((X_test.shape[0], subsequences, timesteps, 1))

In [0]:

X_test_series_sub.shape

In [0]:

model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dropout(0.4))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer=adam)

In [0]:
cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, y_train, batch_size = 4096,epochs = 100)

In [0]:
submission_lstm_cnn = model_cnn_lstm.predict(X_test_series_sub)

In [0]:
submission_lstm_cnn

In [0]:
submission_lstm_cnn = submission_lstm_cnn.clip(0,20)
# creating dataframe with required columns 
submission_lstm_cnn_csv = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_lstm_cnn.ravel()})

# creating csv file from dataframe
submission_lstm_cnn_csv.to_csv('sub_lstm_cnn.csv',index = False)

In [0]:
# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


In [0]:
X_train.shape

In [0]:
my_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)

In [0]:
# creating submission file 
submission_pfs = my_model.predict(X_test)

In [0]:
# we will keep every value between 0 and 20

submission_pfs.shape
submission_pfs

In [0]:
submission_pfs = submission_pfs.clip(0,20)


In [0]:
submission_pfs

In [0]:


# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_pfs.ravel()})

# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)


In [0]:
submission.head()